<a href="https://colab.research.google.com/github/Mo-kw/spacebot-educational-llm/blob/main/LLM_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets


In [ ]:
from datasets import load_dataset
import pandas as pd
import json

# Load SciQ dataset
dataset = load_dataset("allenai/sciq", split="train")

# Filter for space-related keywords
keywords = ["space", "planet", "astronomy", "moon", "star", "galaxy", "universe", "orbit", "gravity", "black hole", "comet", "solar system", "asteroid"]

def is_space_question(q):
    q_lower = q["question"].lower()
    return any(kw in q_lower for kw in keywords)

space_questions = dataset.filter(is_space_question)

print(f"✅ Found {len(space_questions)} space-related questions.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.99M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

✅ Found 406 space-related questions.


In [ ]:
# Format for fine-tuning: instruction–output
converted = []
for item in space_questions:
    converted.append({
        "instruction": item["question"],
        "output": item["correct_answer"]
    })

# Show a few examples
converted[:3]


[{'instruction': 'What galaxy is our solar system a part of?',
  'output': 'milky way'},
 {'instruction': "What man-made devices in space are used to observe the earth's surface?",
  'output': 'satellites'},
 {'instruction': 'What science includes many fields of science related to our home planet?',
  'output': 'earth science'}]

In [ ]:
# Save as JSON
with open("space_dataset.json", "w") as f:
    json.dump(converted, f, indent=2)

# Save as CSV
df = pd.DataFrame(converted)
df.to_csv("space_dataset.csv", index=False)

print("✅ Saved space_dataset.json and space_dataset.csv")


✅ Saved space_dataset.json and space_dataset.csv


In [ ]:
from google.colab import files
files.download("space_dataset.json")
files.download("space_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q transformers accelerate peft bitsandbytes datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import json
from datasets import Dataset

# Load your custom dataset
with open("space_dataset.json", "r") as f:
    data = json.load(f)

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)

# Preview
dataset[0]


{'instruction': 'What galaxy is our solar system a part of?',
 'output': 'milky way'}

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def format_example(example):
    return {"text": f"### Question: {example['instruction']}\n### Answer: {example['output']}"}

formatted_dataset = dataset.map(format_example)

def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])


Map:   0%|          | 0/406 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./trained_phi2_space",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    logging_steps=5,
    save_steps=50,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-2140815095.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,6.878300
10,6.011200
15,3.822800
20,1.202700
25,0.355600
30,0.382900
35,0.303200
40,0.393400
45,0.262100
50,0.206600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=1218, training_loss=0.2385635149889979, metrics={'train_runtime': 901.1461, 'train_samples_per_second': 1.352, 'train_steps_per_second': 1.352, 'total_flos': 4960054479421440.0, 'train_loss': 0.2385635149889979, 'epoch': 3.0})

In [ ]:
import shutil

save_dir = "./final_phi2_space_model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Zip it for download
shutil.make_archive("phi2_space_model", 'zip', save_dir)

print("✅ Model saved and zipped.")


✅ Model saved and zipped.


In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Suppress all Python warnings

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Optional: disable tokenizer parallelism warning
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Path to saved model directory
save_dir = "./final_phi2_space_model"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    save_dir,
    load_in_4bit=True,            # still works, but will be deprecated
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_dir)

print("🪐 SpaceBot is ready! Type 'exit' or 'quit' to leave.\n")

while True:
    question = input("You: ").strip()
    if question.lower() in ["exit", "quit"]:
        print("SpaceBot: Goodbye! 👋")
        break

    prompt = f"### Question: {question}\n### Answer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("### Answer:")[-1].strip()

    max_answer_length = 500
    if len(answer) > max_answer_length:
        answer = answer[:max_answer_length] + "..."

    print(f"SpaceBot: {answer}\n")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🪐 SpaceBot is ready! Type 'exit' or 'quit' to leave.

You: What is the Milky Way?
SpaceBot: a galaxy

You: at planet do humans live on?
SpaceBot: earth

You: What does NASA do?
SpaceBot: explore space

You: exit
SpaceBot: Goodbye! 👋


If the sesion colsed you can upload the model and star chat with it (:

In [ ]:
# Step 1: Install necessary libraries
!pip install -q transformers accelerate peft bitsandbytes

# Optional: Upgrade if needed
!pip install --upgrade transformers accelerate

# Step 2: Unzip the saved model
import zipfile
import os

zip_path = "/content/phi2_space_model.zip"  # or wherever your zip file is
extract_dir = "/content/final_phi2_space_model"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Model unzipped.")

# Step 3: Load the model and tokenizer in 4-bit using BitsAndBytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    extract_dir,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(extract_dir)
tokenizer.pad_token = tokenizer.eos_token

print("✅ Model and tokenizer loaded.")




In [ ]:
# Step 4: Suppress warnings and tokenizer parallelism notice
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Step 5: Run Chat Loop
print("\n🪐 SpaceBot is ready! Type 'exit' or 'quit' to leave.\n")

while True:
    question = input("You: ").strip()
    if question.lower() in ["exit", "quit"]:
        print("SpaceBot: Goodbye! 👋")
        break

    prompt = f"### Question: {question}\n### Answer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("### Answer:")[-1].strip()

    max_answer_length = 500
    if len(answer) > max_answer_length:
        answer = answer[:max_answer_length] + "..."

    print(f"SpaceBot: {answer}\n")


🪐 SpaceBot is ready! Type 'exit' or 'quit' to leave.

You: How many planets are in the solar system?
SpaceBot: 8

You: What is the largest planet?
SpaceBot: jupiter

You: Do astronauts feel gravity in space?
SpaceBot: yes

You: What causes day and night on Earth?
SpaceBot: rotation

You: exit
SpaceBot: Goodbye! 👋
